# Modale Analyse und Antwortspektrumverfahren an einem Rahmen {#sec-antwortspek}

[Aufgabenstellung](calculations/%C3%9Cbung_5_%20Modale_Analyse_2_MMS.pdf)

Dies ist eine Weiterführung des bereits bekannten Rahmentragwerks aus @sec-mms_rahmen .
![Statisches System des Rahmentragwerks](pictures/mms5.jpg){#fig-system}


In [1]:
import sympy as sp 
from sympycalcs import render, convert
import sympy.physics.units as unit

import matplotlib.pyplot as plt

## Parameter der Aufgabenstellung

In [2]:
params = {'E': 30*10**3 *unit.N/unit.mm**2,
          'I':2*10**9*unit.mm**4,
          'm_1':2*20000*unit.N*unit.second**2/unit.m,
          'm_2':20000*unit.N*unit.second**2/unit.m,
          'H':3.2*unit.m,
          
          }

render.dict_render(params)

Eq(E, 30000*newton/millimeter**2)

Eq(I, 2000000000*millimeter**4)

Eq(m_1, 40000*newton*second**2/meter)

Eq(m_2, 20000*newton*second**2/meter)

Eq(H, 3.2*meter)

In [3]:
omega, t,  l, m_1, m_2, E,I, H = sp.symbols('omega, t,  l, m_1, m_2, E, I, H')

omega_n = sp.symbols('omega_n')

## Steifigkeitsmatrix $\mathbf{K}$

Zur Bestimmung der Steifigkeitsmatrix ist das System an jedem Freiheitsgrad auszulenken, wie in @fig-steifigkeit dargestellt ist.

![Auslenkung der Freiheitsgrade zur Bestimmung der Steifigkeit](pictures/steifigkeit_mms2.jpg){#fig-steifigkeit}

Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:
- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

### Horizontale Steifigkeit

Die horizontale Steifigkeit des Systems lässt sich mittels der Formel für Rahmen bestimmen.

$$k_{Stütze} = \frac{12EI_{Stütze}}{H^3}$$

Diese gilt für eine einzelne Stütze.

In [4]:
k_1, k_2 = sp.symbols('k_1, k_2')

params['k_1'] = 2*(12 * 2*E*I / H**3).subs(params).simplify().evalf(6)
params['k_2'] = 2*(12 * E*I / H**3).subs(params).simplify().evalf(6)
K = sp.Matrix([[k_1 + k_2, -k_2],[-k_2, k_2]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

Eq(K, Matrix([
[k_1 + k_2, -k_2],
[     -k_2,  k_2]]))

Eq(K, Matrix([
[ 1.31836e+8*newton/meter, -4.39453e+7*newton/meter],
[-4.39453e+7*newton/meter,  4.39453e+7*newton/meter]]))

## Eigenvektoren
### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem gleichen Aufbau wie die Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen.

In [5]:
m_1, m_2 = sp.symbols('m_1, m_2')
M = sp.Matrix([[m_1, 0],[0, m_2]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

Eq(M, Matrix([
[m_1,   0],
[  0, m_2]]))

Eq(M, Matrix([
[40000*newton*second**2/meter,                            0],
[                           0, 20000*newton*second**2/meter]]))

### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [6]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

Eq(omega_1, 33.1/second)

Eq(omega_2, 66.3/second)

### Eigenvektoren $\phi$

In [7]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_21'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_11 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_11).values())[0]

params['phi_11'] = phi_11.subs(params).simplify() 




render.eq_display((K-omega_1**2 *M)*phi_1,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))

Eq(Matrix([
[-k_2*phi_21 + phi_11*(k_1 + k_2 - m_1*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) - sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))],
[      -k_2*phi_11 + phi_21*(k_2 - m_2*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) - sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_1, Matrix([
[0.5],
[1.0]]))

In [8]:
params['phi_22'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_12 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_12).values())[0]

params['phi_12'] = phi_12.subs(params).simplify() 




render.eq_display((K-omega_2**2 *M)*phi_2,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))

Eq(Matrix([
[-k_2*phi_22 + phi_12*(k_1 + k_2 - m_1*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) + sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))],
[      -k_2*phi_12 + phi_22*(k_2 - m_2*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) + sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_2, Matrix([
[-1.0],
[ 1.0]]))

### Orthogonalitätsbedingung

Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Dies gilt es für die Massenmatrix zu kontrollieren:

$$\mathbf{\phi_1^T M \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_1 = 0}$$

Sowohl auch für die Steifigkeitsmatrix:

$$\mathbf{\phi_1^T K \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_1 = 0}$$



In [9]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

Eq(phi_1.T*M*phi_1, Matrix([[3.0e+4*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_2, Matrix([[6.0e+4*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_1, Matrix([[0]]))

Eq(phi_1.T*M*phi_2, Matrix([[0]]))

In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(4),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(4))

Eq(phi_1.T*K*phi_1, Matrix([[3.3e+7*newton/meter]]))

Eq(phi_2.T*K*phi_2, Matrix([[2.64e+8*newton/meter]]))

Eq(phi_2.T*K*phi_1, Matrix([[0]]))

Eq(phi_1.T*K*phi_2, Matrix([[0]]))

### Eigenformen

![Die beiden Eigenformen skizziert](pictures/eigenformen_mms2.jpg){#fig-eigenformen}

## Modale Analyse

Die Bewegungsgleichung für einen ungedämpften, frei schwingenden Mehrmassenschwinger lässt sich folgend beschreiben:

$$\mathbf{M u''(t) + K u = 0}$$

Die Matrix-Gleichung beschreibt ein System aus Differentialgleichungen. Die Modale Analyse zielt darauf ab, diese zu entkoppeln. Bezogen auf den Mehrmassenschwinger heisst eine Entkoppelung, dass diese in Einmassenschwinger aufgeteilt werden. Dies wird nun schrittweise durchgeführt.

### Modal- und Spektralmatrix

Mittels der Modal- und Spektralmatrix können die generalisierten Grössen ermittelt werden. Diese sind die eigenschaften der einzelnen Einmassenschwinger. Die generalisierten Werte besitzen keine physikalischen Werte, sie sind abhängig von der Wahl der Eigenvektoren, welche bekanntlich von der Normierung abhängen.


Aufgrund der Bewegungsgleichung können die generalisierten Grössen bestimmt werden, es gilt:

$$\mathbf{\Phi^T M \Phi u''(t) + \Phi^T K \Phi u(t) = 0}$$

$$\mathbf{M^*u''(t) + K^* u(t) = 0}$$

Alle $N$-Eigenwerte und alle $N$-Eigenvektoren können kompakt
mit Matrizen ausgedrückt werden:


In [11]:
Phi = sp.Matrix([[phi_1, phi_2]])
Omega = sp.Matrix([[omega_1, 0],[0, omega_2]])

render.eq_display('Modalmatrix', 'Phi',
                  sp.MatrixSymbol('Phi', 2, 2), Phi.subs(params).evalf(4),
                  'Spektralmatrix', 'Omega^2',
                  sp.MatrixSymbol('Omega^2', 2, 2), (Omega**2).subs(params).evalf(4))

Eq(Modalmatrix, Phi)

Eq(Phi, Matrix([
[0.5, -1.0],
[1.0,  1.0]]))

Eq(Spektralmatrix, Omega**2)

Eq(Omega^2, Matrix([
[1099.0/second**2,                0],
[               0, 4395.0/second**2]]))

### Generalisierte Grössen


In [12]:
M_star = Phi.T * M * Phi
K_star = Phi.T * K * Phi


render.eq_display(sp.MatrixSymbol('M^\star', 2,2),sp.simplify(M_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('K^\star', 2,2),sp.simplify(K_star.subs(params)).evalf(5))
                 

Eq(M^\star, Matrix([
[30000.0*newton*second**2/meter,                              0],
[                             0, 60000.0*newton*second**2/meter]]))

Eq(K^\star, Matrix([
[3.2959e+7*newton/meter,                      0],
[                     0, 2.6367e+8*newton/meter]]))

### Kontrolle der modalen Transformation

In [13]:
omega_1_kontrolle = sp.sqrt(K_star[0] / M_star[0])
omega_2_kontrolle = sp.sqrt(K_star[3] / M_star[3])

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_1_modal', omega_1_kontrolle.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(4),
                  'omega_2_modal', omega_2_kontrolle.subs(params).simplify().evalf(4))




Eq(omega_1, 33.1/second)

Eq(omega_1_modal, 33.1/second)

Eq(omega_2, 66.29/second)

Eq(omega_2_modal, 66.29/second)

### Partizipationsfaktor $\Gamma$

Die Verteilung des Partizipationsfaktor gibt einen direkten Hinweis, welcher Eigenmode an der Gesamtanwort den grössten Einfluss (beteiligt bzw. partizipiert) hat.

$$\Gamma_n = \frac{\Phi_n^T \mathbf{M 1}}{\Phi_n^T \mathbf{M}\Phi_n}$$

In allgemeiner Form lautet der Partizipationsfaktor:

$$\Gamma_n = \frac{\Phi_n^T \mathbf{M r^\star}}{\Phi_n^T \mathbf{M}\Phi_n}$$

$\mathbf{r^\star}$ beschreibt die Starrkörperverschiebung infolge der Erdbebenanregung $u_g$ am Fusspunktb des Gesamtsystems.



$$\mathbf{r^\star} = \begin{bmatrix}
FHG_1 \\
FHG_2 
\end{bmatrix}=
\begin{bmatrix}
\cos(0) \\
\cos(0) 
\end{bmatrix}=
\begin{bmatrix}
1 \\
1 
\end{bmatrix}= \mathbf{1}$$


In [14]:
one_vec = sp.ones(2,1)

gamma_1 = (phi_1.T*M*one_vec)[0]/(phi_1.T*M*phi_1)[0]
gamma_2 = (phi_2.T*M*one_vec)[0]/(phi_2.T*M*phi_2)[0]
gamma_sqr = sp.Matrix([[gamma_1**2],[gamma_2**2]])
render.eq_display('Gamma_1', gamma_1,
                  'Gamma_1', gamma_1.subs(params).evalf(3),
                  'Gamma_2', gamma_2,
                  'Gamma_2', gamma_2.subs(params).evalf(3),
                  sp.MatrixSymbol('Gamma^2_n',2,1),gamma_sqr.subs(params).evalf(3))

Eq(Gamma_1, (m_1*phi_11 + m_2*phi_21)/(m_1*phi_11**2 + m_2*phi_21**2))

Eq(Gamma_1, 1.33)

Eq(Gamma_2, (m_1*phi_12 + m_2*phi_22)/(m_1*phi_12**2 + m_2*phi_22**2))

Eq(Gamma_2, -0.333)

Eq(Gamma^2_n, Matrix([
[ 1.78],
[0.111]]))

### Effektive Modale Massen

Durch Multiplikation der Modalen Massen mit dem Partizipationsfaktor resultieren die effektiven Modalen Massen.

In [15]:
m_1_eff = gamma_1**2 * M_star[0]
m_2_eff = gamma_2**2 * M_star[3]

render.eq_display('m_1eff', m_1_eff.subs(params).evalf(3),
                  'm_2eff', m_2_eff.subs(params).evalf(3),
                  'm_tot', (m_1_eff+m_2_eff).subs(params).evalf(3))


Eq(m_1eff, 5.33e+4*newton*second**2/meter)

Eq(m_2eff, 6.67e+3*newton*second**2/meter)

Eq(m_tot, 6.0e+4*newton*second**2/meter)

### Modale Höhen

In [16]:

h_1_star = (H*m_1*phi_11 + H*m_2*phi_12)  / (m_1 * phi_11 + m_2 * phi_12)
h_2_star = (H*m_1*phi_21 + H*m_2*phi_22)  / (m_1 * phi_21 + m_2 * phi_22)

render.eq_display('h_1', h_1_star.subs(params),
                  'h_2', h_2_star.subs(params))

False

Eq(h_2, 3.2*meter)

## Modale Analyse

Die Bewegungsgleichung für einen ungedämpften, periodisch, harmonisch angeregten Mehrmassenschwinger lässt sich folgend beschreiben:

$$\mathbf{M u''(t) + K u = F(t)}$$

Die Matrix-Gleichung beschreibt ein System aus Differentialgleichungen. Die Modale Analyse zielt darauf ab, diese zu entkoppeln. Bezogen auf den Mehrmassenschwinger heisst eine Entkoppelung, dass diese in Einmassenschwinger aufgeteilt werden. Dies wird nun schrittweise durchgeführt.

### Modal- und Spektralmatrix

Mittels der Modal- und Spektralmatrix können die generalisierten Grössen ermittelt werden. Diese sind die eigenschaften der einzelnen Einmassenschwinger. Die generalisierten Werte besitzen keine physikalischen Werte, sie sind abhängig von der Wahl der Eigenvektoren, welche bekanntlich von der Normierung abhängen.


Aufgrund der Bewegungsgleichung können die generalisierten Grössen bestimmt werden, es gilt:

$$\mathbf{\Phi^T M \Phi u''(t) + \Phi^T K \Phi u(t) = \Phi^T F(t)}$$

$$\mathbf{M^*u''(t) + K^* u(t) = F^* (t)}$$

Alle $N$-Eigenwerte und alle $N$-Eigenvektoren können kompakt
mit Matrizen ausgedrückt werden:


In [17]:
Phi = sp.Matrix([[phi_1, phi_2]])
Omega = sp.Matrix([[omega_1, 0],[0, omega_2]])

render.eq_display('Modalmatrix', 'Phi',
                  sp.MatrixSymbol('Phi', 2, 2), Phi.subs(params).evalf(4),
                  'Spektralmatrix', 'Omega^2',
                  sp.MatrixSymbol('Omega^2', 2, 2), (Omega**2).subs(params).evalf(4))

Eq(Modalmatrix, Phi)

Eq(Phi, Matrix([
[0.5, -1.0],
[1.0,  1.0]]))

Eq(Spektralmatrix, Omega**2)

Eq(Omega^2, Matrix([
[1099.0/second**2,                0],
[               0, 4395.0/second**2]]))

### Generalisierte Grössen


In [18]:
M_star = Phi.T * M * Phi
K_star = Phi.T * K * Phi


render.eq_display(sp.MatrixSymbol('M^\star', 2,2),sp.simplify(M_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('K^\star', 2,2),sp.simplify(K_star.subs(params)).evalf(5))

Eq(M^\star, Matrix([
[30000.0*newton*second**2/meter,                              0],
[                             0, 60000.0*newton*second**2/meter]]))

Eq(K^\star, Matrix([
[3.2959e+7*newton/meter,                      0],
[                     0, 2.6367e+8*newton/meter]]))

### Kontrolle der modalen Transformation

In [19]:
omega_1_kontrolle = sp.sqrt(K_star[0] / M_star[0])
omega_2_kontrolle = sp.sqrt(K_star[3] / M_star[3])

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_1_modal', omega_1_kontrolle.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(4),
                  'omega_2_modal', omega_2_kontrolle.subs(params).simplify().evalf(4))




Eq(omega_1, 33.1/second)

Eq(omega_1_modal, 33.1/second)

Eq(omega_2, 66.29/second)

Eq(omega_2_modal, 66.29/second)

## Elastisches Antwortspektrum

Dem Vorgehen nach **SIA261:2020 16.2.3.2** entsprechend, werden folgende Parameter definiert:

- Baugrundklasse B
- Erdbebenzone Z2
- $a_{gd} = 1.0 \text{ m}/\text{s}^2$

In [20]:
T_B = 0.08*unit.second
T_C = 0.35*unit.second
T_D = 2*unit.second
S=1.2
eta = 1.0
a_gd = 1*unit.m/unit.second**2

### Grundschwingzeit

Die Grundschwingzeit kann anhand der bereits ermittelten Eigenkreisfrequenzen ermittelt werden.

In [21]:
T_1 = (omega_1 /(2*sp.pi))**-1
T_2 = (omega_2 /(2*sp.pi))**-1

render.eq_display('T_1', T_1.subs(params).evalf(3),
                  'T_2', T_2.subs(params).evalf(3),
                  )

Eq(T_1, 0.19*second)

Eq(T_2, 0.0948*second)

### Elastisches Antwortspektrum

In [22]:
def antwortspektrum(T_B, T_C, T_D, eta, S, a_gd,T):
    
    if T<=T_B:
        render.eq_display('S_e','a_gd*S_*(1+(2.5*eta_-1)*T/T_B)')
        return a_gd * S *(1+(2.5*eta-1)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_e','2.5*a_gd*S_*eta')
        return 2.5*a_gd*S*eta
    
    if T_C<T<=T_D:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C/T')
        return 2.5*a_gd*S*eta*T_C/T
    
    if T_D<=T:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C*T_D/T**2')
        return 2.5*a_gd*S*eta*T_C*T_D/T**2
    
    else:
        print('Falsche Eingabe')


In [23]:

S_e_1 = antwortspektrum(T_B, T_C, T_D, eta, S, a_gd, T_1.subs(params).evalf(3))
render.eq_display('S_e_1', S_e_1.evalf(3))

Eq(S_e, 2.5*S_*a_gd*eta)

Eq(S_e_1, 3.0*meter/second**2)

In [24]:
S_e_2 = antwortspektrum(T_B, T_C, T_D, eta, S, a_gd, T_2.subs(params).evalf(3))

render.eq_display('S_e_2', S_e_2.evalf(3))

Eq(S_e, 2.5*S_*a_gd*eta)

Eq(S_e_2, 3.0*meter/second**2)

### Bemessungsspektrum

Es wird lediglich auf charakteristischem Niveau die Kräfte bestimmt

In [25]:
q = 1.5
yf = 1.5

In [26]:
def bemessungsspektrum(T_B, T_C, T_D, q, yf, S, a_gd,T):
    g = 9.81*unit.m/unit.second**2
    if T<=T_B:
        render.eq_display('S_d','yf*a_gd/g*S_*(0.67+(2.5/q-0.67)*T/T_B)')
        return yf*a_gd/g*S*(0.67+(2.5/q-0.67)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_d','2.5*yf*a_gd/g*S_/q')
        return 2.5*yf*a_gd/g*S/q
    
    if T_C<T<=T_D:
        render.eq_display('S_e', '2.5*yf*a_gd/g*S_*T_C/(T*q)')
        return 2.5*yf*a_gd/g*S*T_C/(T*q)
    
    if T_D<=T:
        return max([2.5*yf*a_gd/g*S*T_C*T_D/(T**2*q),0.1*yf*a_gd/g])
    
    else:
        print('Falsche Eingabe')

### Einwirkungen

In [27]:
f_11 = gamma_1*m_1_eff*phi_11*S_e_1
f_21 = gamma_1*m_2_eff*phi_21*S_e_1

f_12 = gamma_2*m_1_eff*phi_12*S_e_2
f_22 = gamma_2*m_2_eff*phi_22*S_e_2


f_jn = sp.Matrix([[f_11, f_12],[f_12,f_22]])

render.eq_display(sp.MatrixSymbol('f_jn', 2,2),f_jn.subs(params).evalf(3))

Eq(f_jn, Matrix([
[1.07e+5*newton,  5.33e+4*newton],
[5.33e+4*newton, -6.67e+3*newton]]))

### Deformation

In [28]:
u_11 = gamma_1*phi_11*S_e_1/omega_1**2
u_21 = gamma_1*phi_21*S_e_1/omega_1**2
u_12 = gamma_2*phi_12*S_e_2/omega_2**2
u_22 = gamma_2*phi_22*S_e_2/omega_2**2


u_jn = sp.Matrix([[u_11, u_12],[u_12,u_22]])

render.eq_display(sp.MatrixSymbol('u_jn', 2,2),unit.convert_to(u_jn.subs(params).evalf(3),unit.mm))

Eq(u_jn, Matrix([
[ 1.82*millimeter,  0.228*millimeter],
[0.228*millimeter, -0.228*millimeter]]))

## Überlagerung der EMS

Um die Entkoppelung rückzuführen gilt es die erhaltenen Resultate zu überlagern. Dabei gibt es unterschiedliche Ansätze. Bei weit auseinander liegenden Eigenfrequenzen kann die SRSS-Überlagerung verwendet werden.

$$V_{b2max} = \sqrt{\sum_{n=1}^2 f_{j,n}^2}$$

In [29]:
V_b1_max = sp.sqrt(f_11**2 + f_12**2)
V_b2_max = sp.sqrt(f_21**2 + f_22**2)

render.eq_display('V_b1max',V_b1_max.subs(params).evalf(3),
                  'V_b2max', V_b2_max.subs(params).evalf(3))

Eq(V_b1max, 1.19e+5*newton)

Eq(V_b2max, 2.75e+4*newton)

$$u_{bn} = \sqrt{\sum_{n=1}^2 u_{j,n}^2}$$

In [30]:
u_b1 = sp.sqrt(u_11**2 + u_12**2)
u_b2 = sp.sqrt(u_21**2 + u_22**2)

render.eq_display('u_b1',unit.convert_to(u_b1.subs(params).evalf(3), unit.mm),
                  'u_b2', unit.convert_to(u_b2.subs(params).evalf(3),unit.mm))

Eq(u_b1, 1.83*millimeter)

Eq(u_b2, 3.65*millimeter)